# Implementation of *innate training* (post-training)

1. Overview
2. Purpose
3. Preparation
4. Implementation
    - Timer task
5. Demonstrations

## 1. Overview
Here, through several tasks, let's evaluate the information processing capability of the pre-trained network obtained in the previous notebook (`innate_training_pre.ipynb`).


## 2. Purpose
We are implementing an online training setup with innate training and replicating the paper's results by exploiting high-dimensional chaotic time series. 

## 3. Preparation
Just run the following importing scripts (press `shift+Enter` to run each cell).
You will see an authentification popup, then please accept it.

In [ ]:
import os
import sys
import json
import math
import joblib
import numpy as np

from scipy.stats import pearsonr
from collections import defaultdict

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/closed_loop_control/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells

from utils.tqdm import trange, tqdm  # progress bar
from utils.reservoir import Module, ESN  # class for echo state network
from utils.interpolate import interp1d  # interpolation
from utils.style_config import plt
from views.innate_training import *  # visualization wrapper
from tests.innate_training import *  # check function

## 4. Implementatation
### 4.1 Load pre-trained network

Run the following cell to load pre-trained network obtained in `innate_trainining_pre.ipynb`.

In [ ]:
%load_and_run ./answer/rls_update.py
%load_and_run ./answer/innate_esn.py
%load_and_run ./answer/innate_emulate.py

load_dir = "./output"

seed = 1234
rnd = np.random.RandomState(seed)

net_init = InnateESN.read_pickle(f"{load_dir}/net_init.pkl")
net_term = InnateESN.read_pickle(f"{load_dir}/net_term.pkl")
w_in = np.load(f"{load_dir}/w_in.npy")
with open(f"{load_dir}/params.json", mode="r") as f:
    params = json.load(f)

pulse_amp = params["pulse_amp"]
pulse_period = params["pulse_period"]
innate_period = params["innate_period"]
washout_period, record_period = 1000, 10000
w_in_num = w_in.shape[0]
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]
innate_range = [0, innate_period]
plot_range = list(range(5))


def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_in[:w_in_num]
    else:
        return 0.0
f_in.dim = w_in_num

    
def sample_network(record_range, net, f_in, sample_num=5, rnd=None):
    rnd = rnd if rnd else np.random
    net.reset(rnd.uniform(low=-1, high=1, size=(sample_num, net.dim)))
    rec_t, rec_net = emulate(
        np.arange(*record_range), net, f_in=f_in, prefix="sample ")
    return (rec_t, rec_net)


In [ ]:
rec_sample = sample_network(record_range, net_init, f_in, rnd=rnd)
fig = show_trajectory(*rec_sample, plot_range, lw=1.0, title="initial",
                      pulse_range=pulse_range, innate_range=innate_range)

rec_sample = sample_network(record_range, net_term, f_in, rnd=rnd)
fig = show_trajectory(*rec_sample, plot_range, lw=1.0, title="pre-trained",
                      pulse_range=pulse_range, innate_range=innate_range)

### 4.2 Timer task

Here, let's introduce a benchmark task called *timer task*, which is originally introduced in [R. Laje et al. (2013)](https://www.nature.com/articles/nn.3405).
The timer task evaluates the temporal information processing capability of the system where the additional linear open-loop readout is tuned to output pulse-like wave accurately after a certain period of time $t_\text{peak}$.
This optimization is done by the Ridge regression, which was implemented in the previous notebook.

The performance is normally evaluated by the $R^2$ score, which can take 1.0 at the maximum corresponding to the complete reconstruction.
By integrating $R^2$ score over $t_\text{pulse}$ until it takes to 0, we can define *timer-task capacity*, which quantitatively evaluates the degree of memory locally storing the input events.

Please fill blanks in the following cell implementing the timer task.

Specification of `train_timer_task_model`
- Arguments
    - `ts` (`ndarray` or `list`): List of time steps (*e. g.*, $[0, 1, 2, ..., T-1]$).
    - `xs` (`ndarray`): Sampled ESN's dynamics with different initial conditions (*i. e.*, Shape `(len(ts), batch_size, net.dim)`).
    - `t_slice` (`slice` or `list` or `None`): Time range of training dataset (*e. g.,* `slice(0, 1000)`).
    - `t_peak` (`float`): Peak of the target pulse.
    - `t_sigma` (`float`): Width of the target pulse.
    - `alpha` (`float`): Regularization term of ridge regression.
- Return
    - `np.ndarray`: Difference of matrix ($\Delta W$).

<details>
<summary>
Hints
</summary>

- Please use `RidgeReadout`.
- You can also `Ridge`. Please read the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [ ]:
%load_and_run ./answer/linear.py


def pulse_func(ts, t_peak, t_sigma):
    return np.exp(-(ts - t_peak)**2 / (2 * t_sigma**2))


def train_timer_task_model(ts, xs, t_slice, t_pulse, t_sigma=10.0, alpha=1.0):
    assert xs.ndim == 3
    _t_len, batch_size, net_dim = xs.shape
    X_train = xs[t_slice].reshape(-1, net_dim)
    Y_train = pulse_func(ts[t_slice], t_pulse, t_sigma)
    Y_train = np.repeat(Y_train, batch_size)
    Y_train = Y_train.reshape(-1, 1)
    # TODO implement Ridge regression
    ...
    return model


def eval_timer_task_model(ts, xs, model, t_slice, t_peak, t_sigma=10.0, alpha=1.0):
    t_len, batch_size, net_dim = xs.shape
    Y_out = model.predict(xs.reshape(-1, net_dim)).reshape(t_len, batch_size)
    Y_eval = pulse_func(ts, t_peak, t_sigma)
    r_sq = np.array([pearsonr(Y_eval[t_slice], Y_out[t_slice, idx])[0] for idx in range(batch_size)])
    return Y_out, Y_eval, r_sq

## 5. Demonstration

Let's evaluate the timer-task performance and capacities.

First, sample the training and evaluation data (it will takes 10~20 seconds). 

In [ ]:
train_num, eval_num = 10, 5
ts_init, xs_init = sample_network(
    [-washout_period, record_period], net_init, f_in, sample_num=train_num + eval_num)
ts_term, xs_term = sample_network(
    [-washout_period, record_period], net_term, f_in, sample_num=train_num + eval_num)

### 5.1 Timer-task performance (*e. g.,* $t_\text{peak} = 1500$)

Compare the performance of the initial and pre-trained network.

In [ ]:
def show_performance(ts, xs, t_peak, title=None):
    t_slice = slice(washout_period, washout_period + t_peak + 50)
    model = train_timer_task_model(
        ts, xs[:, :train_num], t_slice, t_peak)
    Y_out, Y_eval, r_sq = eval_timer_task_model(
        ts, xs[:, train_num:], model, t_slice, t_peak)
    fig = show_output(
        ts, xs[:, train_num:], Y_out, Y_eval, 
        title=r"{} ($R^2={:.2f}$)".format(title, r_sq[-1].mean()))
    fig[0].set_xlim([-washout_period, t_peak + 2000])
    fig[0].fill_x(0, t_peak, facecolor="cyan", alpha=0.2)
    fig[1].set_xlim([-washout_period, t_peak + 2000])
    fig[1].fill_x(0, t_peak, facecolor="cyan", alpha=0.2)
    return fig

fig = show_performance(ts_init, xs_init, 1500, "initial")
fig = show_performance(ts_term, xs_term, 1500, "pre-trained")


### 5.2 Timer-task capacity ($t_\text{peak} = 100, 200, \cdots, 10000$)

Let's change $t_peak$ and how the performance decreases by increasing it.

In [ ]:
def eval_capacity(ts, xs, t_peaks, title=None):
    record = None
    for idx, t_peak in enumerate(tqdm(t_peaks)):
        t_slice = slice(washout_period, washout_period + t_peak + 50)
        model = train_timer_task_model(ts, xs[:, :train_num], t_slice, t_peak)
        _out, _eval, r_sq = eval_timer_task_model(ts, xs[:, train_num:], model, t_slice, t_peak)
        if record is None:
            record = np.zeros((len(t_peaks), *r_sq.shape))
        record[idx] = r_sq
    return record

t_peaks = np.arange(1, 101) * 100
    
capacities = {}
capacities["initial"] = eval_capacity(ts_init, xs_init, t_peaks)
capacities["pre-trained"] = eval_capacity(ts_term, xs_term, t_peaks)
    
fig = Figure(figsize=(8, 6))
for name, cap in capacities.items(): 
    fig[0].fill_std(t_peaks, cap.mean(axis=-1), cap.std(axis=-1), label=name)
    fig[0].line_y(1.0, ls=":", lw=1.0, color="k")
    fig[0].line_y(0.0, ls=":", lw=1.0, color="k")
fig[0].legend()
None

The area between $R^2=0$ and $1$ represents the timer task capacity. 
You can understand that the pre-training process significantly increases the system's capacity.

## Advanced
- Evaluate the node dependency.
- Check what occurs with a long innate trajectory.
- Increase `w_in_dim` in the previous notebook and check what happens.